In [ ]:
"""
自定义 TextFileStore 的示例用法
"""

from langgraph.store.base import (
    BaseStore,
    Item,
    SearchItem,
    Op,
    Result,
    IndexConfig,
    NamespacePath,
)
from typing import Optional, Union, Literal, Any, Iterable, List
import json, os, uuid, datetime


class TextFileStore(BaseStore):

    def __init__(self, base_dir: str):
        self.base_dir = base_dir

    def put(
        self,
        namespace: NamespacePath,
        key: str,
        value: dict[str, Any],
        index: Optional[Union[Literal[False], list[str]]] = None,
    ) -> None:
        namespace_path = os.path.join(self.base_dir, *namespace)
        os.makedirs(namespace_path, exist_ok=True)
        file_path = os.path.join(namespace_path, f"{key}.json")
        with open(file_path, "w") as f:
            json.dump(value, f)

    def get(self, namespace: NamespacePath, key: str) -> Optional[Item]:
        file_path = os.path.join(self.base_dir, *namespace, f"{key}.json")
        if not os.path.exists(file_path):
            return None
        with open(file_path, "r") as f:
            value = json.load(f)
        created_at = datetime.datetime.fromtimestamp(
            os.path.getctime(file_path), tz=datetime.timezone.utc
        )
        updated_at = datetime.datetime.fromtimestamp(
            os.path.getmtime(file_path), tz=datetime.timezone.utc
        )
        return Item(
            namespace=namespace,
            key=key,
            value=value,
            created_at=created_at,
            updated_at=updated_at,
        )


file_store = TextFileStore(base_dir="./my_filestore")
file_store.put(
    ("user_data"), "user_profile_1", {"name": "自定义存储用户", "preference": "files"}
)
retrieved_item = file_store.get(("user_data"), "user_profile_1")
if retrieved_item:
    print(f"检索到的条目：{retrieved_item.value}")